In [ ]:
# token の設定を忘れないでね

In [ ]:
from PIL import Image
from matplotlib import cm
import numpy as np
from io import BytesIO
import sys
import zipfile

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# 画像データを追加しましょう

In [ ]:
#  画像のzip ファイルからデータセットを作成する関数
# 画像のサイズを１２８*１28にする
def zip2array(zip,tag):
    array_list = []
    tag_list = []
    with zipfile.ZipFile(BytesIO(zip.read()), 'r') as zfile:
        nWaitTime = 1
        for name in zfile.namelist():
            if '.JPG' not in name or '__' in name:
                continue
            with zfile.open(name,mode='r') as image_file:
                print(name)
                content = image_file.read()
                bytes_io = bytearray(content)
                img = Image.open(BytesIO(bytes_io)).convert('L')
                img_resize = img.resize((128, 128),Image.LANCZOS)
                img_resize.show()
                img_array = np.array(img_resize)
                array_list.append(img_array)
                tag_list.append(tag)
#                 im = Image.fromarray(img_array.astype('uint8'), 'RGB')
#                 im.show()
    return array_list,tag_list

In [ ]:
#  データセットを作成する
#  タグ：　latte--0， chitose--1， cho--2

# 学習用データセット
latte_X_train,latte_Y_train = zip2array([streaming_bodyをここにペスト],0)
chitose_X_train,chitose_Y_train = zip2array([streaming_bodyをここにペスト],1)
cho_X_train,cho_Y_train = zip2array([streaming_bodyをここにペスト],2)
X_train = latte_X_train + chitose_X_train+cho_X_train
Y_train = latte_Y_train + chitose_Y_train+cho_Y_train

X_train = np.asarray(X_train)
Y_train = np.asarray(Y_train) 


# 　テスト用データセット
latte_X_test,latte_Y_test = zip2array(streaming_body_4,0)
chitose_X_test,chitose_Y_test = zip2array(streaming_body_5,1)
cho_X_test,cho_Y_test = zip2array(streaming_body_10,2)
X_test = latte_X_test+ chitose_X_test+cho_X_test
Y_test = latte_Y_test + chitose_Y_test+cho_Y_test

X_test = np.asarray(X_test)
Y_test = np.asarray(Y_test) 

#  タグを確認する
print(Y_train)
print(Y_test)

In [ ]:
class_names =['latte','chitose','cho']

In [ ]:
X_train.shape

In [ ]:
len(Y_train)

In [ ]:
X_train= X_train / 255.0

X_test = X_test / 255.0

In [ ]:
# 　モデルを定義する
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(128, 128)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(3)
])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, Y_train, epochs=30)

In [ ]:
test_loss, test_acc = model.evaluate(X_test,  Y_test, verbose=2)

print('\nTest accuracy:', test_acc)

In [ ]:
#  モデルをテストしましょう
probability_model = tf.keras.Sequential([model, 
                                         tf.keras.layers.Softmax()])

In [ ]:
predictions = probability_model.predict(X_test)

In [ ]:
predictions[6]

In [ ]:
np.argmax(predictions[0])

In [ ]:
# 　テスト画像の分類結果をグラフで見るために作った関数
def plot_image(i, predictions_array, true_label, img):
  predictions_array, true_label, img = predictions_array, true_label[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img, cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
  predictions_array, true_label = predictions_array, true_label[i]
  plt.grid(False)
  plt.xticks(range(10))
  plt.yticks([])
  thisplot = plt.bar(range(3), predictions_array, color="#777777")
  plt.ylim([0, 1])
  predicted_label = np.argmax(predictions_array)

  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')

In [ ]:
num_rows = 3
num_cols = 3
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_image(i, predictions[i], Y_test, X_test)
  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  plot_value_array(i, predictions[i], Y_test)
plt.tight_layout()
plt.show()